# Smart Trash Sorter – Initial Exploration & Model Training
**ITAI 1378: Computer Vision and AI**  
Author: Hieu Lu

This notebook prepares the dataset, trains a MobileNetV3 model, and evaluates its performance for the Smart Trash Sorter project.

In [ ]:
# 🔧 1. Environment Setup
!pip install tensorflow tensorflow-hub tensorflow-datasets tensorflow-lite-support roboflow opencv-python matplotlib pandas scikit-learn tqdm -q
import os, numpy as np, pandas as pd, tensorflow as tf, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print('TensorFlow version:', tf.__version__)

In [ ]:
# 📦 2. Dataset Import
# Option 1: Roboflow dataset (replace API key)
# from roboflow import Roboflow
# rf = Roboflow(api_key='YOUR_API_KEY')
# project = rf.workspace('your-workspace').project('waste-classification')
# dataset = project.version(1).download('folder')

# Option 2: Local directory
data_dir = '/content/waste_dataset'
if not os.path.exists(data_dir): os.makedirs(data_dir)
print('Dataset folder:', data_dir)

In [ ]:
# 🧠 3. Data Preprocessing
img_height, img_width, batch_size = 224, 224, 32
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=15, width_shift_range=0.1,
                                   height_shift_range=0.1, horizontal_flip=True, validation_split=0.2)
train_ds = train_datagen.flow_from_directory(data_dir, target_size=(img_height, img_width), batch_size=batch_size, subset='training')
val_ds = train_datagen.flow_from_directory(data_dir, target_size=(img_height, img_width), batch_size=batch_size, subset='validation')
class_names = list(train_ds.class_indices.keys())
print('Classes:', class_names)

In [ ]:
# 🧩 4. Model Setup – MobileNetV3
base_model = tf.keras.applications.MobileNetV3Small(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 🚀 5. Model Training
EPOCHS = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
# 📈 6. Evaluation
acc, val_acc = history.history['accuracy'], history.history['val_accuracy']
loss, val_loss = history.history['loss'], history.history['val_loss']
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(acc, label='Train'); plt.plot(val_acc, label='Val'); plt.legend(); plt.title('Accuracy')
plt.subplot(1,2,2); plt.plot(loss, label='Train'); plt.plot(val_loss, label='Val'); plt.legend(); plt.title('Loss')
plt.show()

In [ ]:
# 📊 7. Confusion Matrix
val_ds.reset()
Y_pred = model.predict(val_ds)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(val_ds.classes, y_pred, target_names=class_names))
cm = confusion_matrix(val_ds.classes, y_pred)
plt.imshow(cm, cmap='Blues'); plt.title('Confusion Matrix'); plt.xlabel('Predicted'); plt.ylabel('True'); plt.show()

In [ ]:
# 💾 8. Save Model
model.save('/content/smart_trash_sorter_mobilenetv3.h5')
print('Model saved successfully!')